In [29]:
import subprocess
for i in range(2541347, 2541362):
    subprocess.run(["scancel", str(i)])

scancel: error: Kill job error on job id 2541361: Access/permission denied


In [51]:
import re
import subprocess
import time
import os
from random import randrange 
import glob 


file = ""
filename = f"run_script.sh"

with open(filename) as f:
    file = f.read()

result_dir_name = 'results_clip32'

batch_sizes = [256]
lrs = [5e-5]
projection_dims = [512]
num_layerss = [12]
num_headss = [8]
epochss = [100]
warmups = [10]
validation_epochss = [5]
preembed_nodess = [True, False]
text_encoders = ['clip']
transformers = ['ours']


exps = []
for lr in lrs:
    for batch_size in batch_sizes:
        for num_layers in num_layerss:
            for num_heads in num_headss:
                for epochs in epochss:
                    for warmup in warmups:
                        for validation_epochs in validation_epochss:
                            for preembed_nodes in preembed_nodess:
                                for text_encoder in text_encoders:
                                    for transformer in transformers:
                                        for projection_dim in projection_dims:
                                            exp_name = f"{lr}-{num_layers}-{text_encoder}{'_preembed' if preembed_nodes else ''}"
                                            # exp_name = f"{lr}-{text_encoder}{'_preembed' if preembed_nodes else ''}_clip_transformer_prelin"
                                            exps.append(exp_name)
                                            
                                            result_dir = f"{result_dir_name}/{exp_name}"
                                            if not os.path.exists(result_dir):
                                                os.makedirs(result_dir)

                                            file = re.sub('--main_process_port .*', f"--main_process_port {randrange(0, 65535)} " + r"\\" , file)

                                            file = re.sub('--lr .*', f"--lr {lr} " + r"\\" , file)
                                            file = re.sub('--batch_size .*', f"--batch_size {batch_size} " + r"\\" , file)
                                            file = re.sub('--epochs .*', f"--epochs {epochs} " + r"\\" , file)
                                            file = re.sub('--validation_epochs .*', f"--validation_epochs {validation_epochs} " + r"\\" , file)
                                            file = re.sub('--warmup .*', f"--warmup {warmup} " + r"\\" , file)
                                            file = re.sub('--projection_dim .*', f"--projection_dim {projection_dim} " + r"\\" , file)
                                            file = re.sub('--num_heads .*', f"--num_heads {num_heads} " + r"\\" , file)
                                            file = re.sub('--num_layers .*', f"--num_layers {num_layers} " + r"\\" , file)
                                            file = re.sub('--text_encoder .*', f"--text_encoder {text_encoder} " + r"\\" , file)
                                            file = re.sub('--transformer .*', f"--transformer {transformer} " + r"\\" , file)
                                            file = re.sub('--exp_name .*', f"--exp_name {exp_name} " + r"\\" , file)
                                            file = re.sub('--result_dir .*', f"--result_dir {result_dir_name} " + r"\\" , file)

                                            file = re.sub('--job-name=.*', f"--job-name={batch_size}{lr}", file)
                                            file = re.sub('--output .*', f"--output {result_dir}/train.log", file)
                                            file = re.sub('--error .*', f"--error {result_dir}/train.log", file)

                                            if preembed_nodes and "--preembed_nodes \\" not in file:
                                                file += "\n--preembed_nodes \\"
                                            elif preembed_nodes == False and "--preembed_nodes \\" in file:
                                                split = file.split("\n--preembed_nodes \\")
                                                file = "".join(split)

                                            print(file)

                                            with open(filename, 'w') as f:
                                                f.write(file)

                                            subprocess.run(["sbatch", filename])

#!/bin/bash
#SBATCH --job-name=2565e-05
#SBATCH --output results_clip32/5e-05-12-clip_preembed/train.log
#SBATCH --error results_clip32/5e-05-12-clip_preembed/train.log
#SBATCH --time=71:00:00
#SBATCH --gres=gpu:rtxa5000:2
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=64G

accelerate launch --multi_gpu \
--main_process_port 9739 \
contrastive_train.py \
--dataset coco \
--batch_size 256 \
--epochs 100 \
--warmup 10 \
--validation_epochs 5 \
--checkpoint_epochs 20 \
--projection_dim 512 \
--lr 5e-05 \
--num_heads 8 \
--num_layers 12 \
--text_encoder clip \
--transformer ours \
--result_dir results_clip32 \
--exp_name 5e-05-12-clip_preembed \
--preembed_nodes \
Submitted batch job 2542550
#!/bin/bash
#SBATCH --job-name=2565e-05
#SBATCH --output results_clip32/5e-05-12-clip/train.log
#SBATCH --error results_clip32/5e-05-12-clip/train.log
#SBATCH --time=71:00:00
#SBATCH --gres=gpu:rtxa5000:2
#SBATCH --qos=scavenge

In [4]:
import re
import subprocess
import time
import os
from random import randrange 
import glob 


file = ""
filename = f"run_base.sh"

with open(filename) as f:
    file = f.read()

result_dir_name = 'results_clip32'

batch_sizes = [256]
lrs = [1e-5, 5e-5, 5e-6, 1e-6]
projection_dims = [512]
epochss = [100]
warmups = [10]
validation_epochss = [5]
checkpoint_epochs = 5
image_encoders = ['clip']
text_encoders = ['clip']


exps = []
for lr in lrs:
    for batch_size in batch_sizes:
        for epochs in epochss:
            for warmup in warmups:
                for image_encoder in image_encoders:
                    for text_encoder in text_encoders:
                        for validation_epochs in validation_epochss:
                            for projection_dim in projection_dims:
                                exp_name = f"{lr}-{image_encoder}-{text_encoder}_baseline"
                                exps.append(exp_name)
                                
                                result_dir = f"{result_dir_name}/{exp_name}"
                                if not os.path.exists(result_dir):
                                    os.makedirs(result_dir)

                                file = re.sub('--lr .*', f"--lr {lr} " + r"\\" , file)
                                file = re.sub('--batch_size .*', f"--batch_size {batch_size} " + r"\\" , file)
                                file = re.sub('--epochs .*', f"--epochs {epochs} " + r"\\" , file)
                                file = re.sub('--validation_epochs .*', f"--validation_epochs {validation_epochs} " + r"\\" , file)
                                file = re.sub('--checkpoint_epochs .*', f"--checkpoint_epochs {checkpoint_epochs} " + r"\\" , file)
                                file = re.sub('--projection_dim .*', f"--projection_dim {projection_dim} " + r"\\" , file)
                                file = re.sub('--warmup .*', f"--warmup {warmup} " + r"\\" , file)
                                file = re.sub('--image_encoder .*', f"--image_encoder {image_encoder} " + r"\\" , file)
                                file = re.sub('--text_encoder .*', f"--text_encoder {text_encoder} " + r"\\" , file)
                                file = re.sub('--exp_name .*', f"--exp_name {exp_name} " + r"\\" , file)
                                file = re.sub('--result_dir .*', f"--result_dir {result_dir_name} " + r"\\" , file)

                                file = re.sub('--job-name=.*', f"--job-name={batch_size}{lr}", file)
                                file = re.sub('--output .*', f"--output {result_dir}/train.log", file)
                                file = re.sub('--error .*', f"--error {result_dir}/train.log", file)

                                print(file)

                                with open(filename, 'w') as f:
                                    f.write(file)

                                subprocess.run(["sbatch", filename])

#!/bin/bash
#SBATCH --job-name=2561e-05
#SBATCH --output results_clip32/1e-05-clip-clip_baseline/train.log
#SBATCH --error results_clip32/1e-05-clip-clip_baseline/train.log
#SBATCH --time=71:00:00
#SBATCH --gres=gpu:rtxa5000:4
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=32G

python3 contrastive_train_baseline.py \
--dataset coco \
--batch_size 256 \
--epochs 100 \
--warmup 10 \
--validation_epochs 5 \
--checkpoint_epochs 5 \
--projection_dim 512 \
--image_encoder clip \
--text_encoder clip \
--lr 1e-05 \
--exp_name 1e-05-clip-clip_baseline \
Submitted batch job 2518688
#!/bin/bash
#SBATCH --job-name=2565e-05
#SBATCH --output results_clip32/5e-05-clip-clip_baseline/train.log
#SBATCH --error results_clip32/5e-05-clip-clip_baseline/train.log
#SBATCH --time=71:00:00
#SBATCH --gres=gpu:rtxa5000:4
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH -

In [17]:
import re
import subprocess
import glob

file = ""
filename = f"run_aro.sh"

with open(filename) as f:
    file = f.read()

result_dir_name = 'results_clip32'
exps = [
    # 'clip32-clip32_baseline',
    # '1e-06-clip-clip_baseline',
    # '5e-06-clip-clip_baseline',
    # '1e-05-clip-clip_baseline',
    # '4-clip',
    # '4-clip_preembed',
    # '6-clip',
    # '6-clip_preembed',
    # '8-clip',
    # '8-clip_preembed',
    # '5e-06-8-clip',
    # '1e-06-8-clip',
    # '1e-05-8-clip',
    # '12-clip',
    # '12-clip_preembed',
]

exps = [f.split('/')[1] for f in glob.glob(f'{result_dir_name}/*_clip_transformer')]
exps += [f.split('/')[1] for f in glob.glob(f'{result_dir_name}/*_baseline')]

projection_dim = 512
image_encoder = 'clip'
text_encoder = 'clip'
num_layers = 8
datasets = ['aro_vgr', 'aro_vga', 'aro_coco_order']

for exp in exps:
    for dataset in datasets:
        result_dir = f'{result_dir_name}/{exp}'

        num_layers = re.findall(r'-(\d)-', exp)
        if len(num_layers) > 0:
            num_layers = num_layers[0]
        else:
            num_layers = 6
        transformer = 'clip' if '_clip_transformer' in exp else 'ours'
        preembed_nodes = '_preembed' in exp


        file = re.sub('--dataset .*', f"--dataset {dataset} " + r"\\" , file)
        file = re.sub('--projection_dim .*', f"--projection_dim {projection_dim} " + r"\\" , file)
        file = re.sub('--text_encoder .*', f"--text_encoder {text_encoder} " + r"\\" , file)
        file = re.sub('--image_encoder .*', f"--image_encoder {image_encoder} " + r"\\" , file)
        file = re.sub('--transformer .*', f"--transformer {transformer} " + r"\\" , file)
        file = re.sub('--num_layers .*', f"--num_layers {num_layers} " + r"\\" , file)
        file = re.sub('--exp_name .*', f"--exp_name {exp} " + r"\\" , file)

        file = re.sub('--job-name=.*', f"--job-name={dataset}", file)
        file = re.sub('--output .*', f"--output {result_dir}/{dataset}.log", file)
        file = re.sub('--error .*', f"--error {result_dir}/{dataset}.log", file)

        if preembed_nodes and "--preembed_nodes \\" not in file:
            file += "\n--preembed_nodes \\"
        elif preembed_nodes == False and "--preembed_nodes \\" in file:
            split = file.split("\n--preembed_nodes \\")
            file = "".join(split)

        print(file)

        with open(filename, 'w') as f:
            f.write(file)

        subprocess.run(["sbatch", filename])


#!/bin/bash
#SBATCH --job-name=aro_vgr
#SBATCH --output results_clip32/5e-06-clip_preembed_clip_transformer/aro_vgr.log
#SBATCH --error results_clip32/5e-06-clip_preembed_clip_transformer/aro_vgr.log
#SBATCH --time=05:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=16G

python3 aro_eval.py \
--dataset aro_vgr \
--projection_dim 512 \
--image_encoder clip \
--text_encoder clip \
--transformer clip \
--num_layers 6 \
--exp_name 5e-06-clip_preembed_clip_transformer \
--preembed_nodes \
Submitted batch job 2529993
#!/bin/bash
#SBATCH --job-name=aro_vga
#SBATCH --output results_clip32/5e-06-clip_preembed_clip_transformer/aro_vga.log
#SBATCH --error results_clip32/5e-06-clip_preembed_clip_transformer/aro_vga.log
#SBATCH --time=05:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --

In [18]:
import re
import subprocess


file = ""
filename = f"run_winoground.sh"

with open(filename) as f:
    file = f.read()

result_dir_name = 'results_clip32'
exps = [
    # 'clip32-clip32_baseline',
    # '1e-06-clip-clip_baseline',
    # '5e-06-clip-clip_baseline',
    # '1e-05-clip-clip_baseline',
    # '4-clip',
    # '4-clip_preembed',
    # '6-clip',
    # '6-clip_preembed',
    # '8-clip',
    # '8-clip_preembed',
    # '5e-06-8-clip',
    # '1e-06-8-clip',
    # '1e-05-8-clip',
    # '12-clip',
    # '12-clip_preembed',
]

exps = [f.split('/')[1] for f in glob.glob('results_clip32/*_clip_transformer')]
exps += [f.split('/')[1] for f in glob.glob('results_clip32/*_baseline')]

projection_dim = 512
image_encoder = 'clip'
text_encoder = 'clip'
num_layers = 8

for exp in exps:
    result_dir = f'{result_dir_name}/{exp}'

    num_layers = re.findall(r'-(\d)-', exp)
    if len(num_layers) > 0:
        num_layers = num_layers[0]
    else:
        num_layers = 6
    transformer = 'clip' if '_clip_transformer' in exp else 'ours'
    preembed_nodes = '_preembed' in exp


    file = re.sub('--projection_dim .*', f"--projection_dim {projection_dim} " + r"\\" , file)
    file = re.sub('--text_encoder .*', f"--text_encoder {text_encoder} " + r"\\" , file)
    file = re.sub('--image_encoder .*', f"--image_encoder {image_encoder} " + r"\\" , file)
    file = re.sub('--transformer .*', f"--transformer {transformer} " + r"\\" , file)
    file = re.sub('--num_layers .*', f"--num_layers {num_layers} " + r"\\" , file)
    file = re.sub('--exp_name .*', f"--exp_name {exp} " + r"\\" , file)

    file = re.sub('--job-name=.*', f"--job-name=winoground", file)
    file = re.sub('--output .*', f"--output {result_dir}/winoground.log", file)
    file = re.sub('--error .*', f"--error {result_dir}/winoground.log", file)

    if preembed_nodes and "--preembed_nodes \\" not in file:
        file += "\n--preembed_nodes \\"
    elif preembed_nodes == False and "--preembed_nodes \\" in file:
        split = file.split("\n--preembed_nodes \\")
        file = "".join(split)

    print(file)

    with open(filename, 'w') as f:
        f.write(file)

    subprocess.run(["sbatch", filename])


#!/bin/bash
#SBATCH --job-name=winoground
#SBATCH --output results_clip32/5e-06-clip_preembed_clip_transformer/winoground.log
#SBATCH --error results_clip32/5e-06-clip_preembed_clip_transformer/winoground.log
#SBATCH --time=05:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=16G

python3 winoground_eval.py \
--projection_dim 512 \
--image_encoder clip \
--text_encoder clip \
--transformer clip \
--num_layers 6 \
--exp_name 5e-06-clip_preembed_clip_transformer \
--preembed_nodes \
Submitted batch job 2530180
#!/bin/bash
#SBATCH --job-name=winoground
#SBATCH --output results_clip32/1e-06-clip_preembed_clip_transformer/winoground.log
#SBATCH --error results_clip32/1e-06-clip_preembed_clip_transformer/winoground.log
#SBATCH --time=05:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBAT